### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [33]:
# !git clone https://github.com/kstathou/vector_engine

In [34]:
# cd vector_engine

In [35]:
# pip install -r requirements.txt

### Let's begin!

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
# Used to import data from S3.
import pandas as pd
import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]


Stored and processed data in s3

In [17]:
# Use pandas to read files from S3 buckets!
df = pd.read_csv('../data/employee_dummy.csv')

In [18]:
df.columns

Index(['Title', 'Skills', 'year', 'id'], dtype='object')

In [19]:
print(f"Misinformation, disinformation and fake news papers: {df.id.unique().shape[0]}")

Misinformation, disinformation and fake news papers: 299


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [20]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [21]:
# Convert abstracts to vectors
#df.Skills.to_list()[:301]
embeddings = model.encode(df.Skills.to_list(), show_progress_bar=True)

In [22]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [24]:


# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")



Number of vectors in the Faiss index: 299


### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [25]:
# Paper abstract
df.iloc[200, 1]

'Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'

In [26]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[200]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

MAG paper IDs: [2156683801, 2012373852, 1999931407, 2124861983, 171007782, 2060919843, 2000297003, 1974143004, 1972193616, 2943415491]


In [27]:
# Fetch the paper titles based on their index
id2details(df, I, 'Skills')

[['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  

In [28]:
# Fetch the paper abstracts based on their index
id2details(df, I, 'Skills')

[['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux'],
 ['Golang  Python  


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [29]:
user_query = """
Golang  Python  C++  Java  JavaScript  C#  CSS     C  Docker  Keras  Numpy  Pandas  CI  Embedded Linux
"""

In [30]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11, 8.13042827507715e-11]

MAG paper IDs: [2156683801, 2012373852, 1999931407, 2124861983, 171007782, 2060919843, 2000297003, 1974143004, 1972193616, 2943415491]


In [31]:
# Fetching the paper titles based on their index
id2details(df, I, 'id')

[[2156683801],
 [2012373852],
 [1999931407],
 [2124861983],
 [171007782],
 [2060919843],
 [2000297003],
 [1974143004],
 [1972193616],
 [2943415491]]

In [32]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[1]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index4.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/home/mehdi/repos/vector_engine
